In [ ]:
import numpy as np
import random
from numba.typed import List as NumbaList
from BPnumba.NumAG import InstancePob,TrainAG
from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from BPnumba.NumFun import CreatePoblation
from numba import types, prange,njit,deferred_type
from typing import List
from collections import OrderedDict
from numba.experimental import jitclass
from BPnumba.DFFAnumba import DFFtrain


In [ ]:
P2A2 = PROBLEM2[1]
#contenedor = UDLPROBLEM["UDL2"]["dimension"]
#datos =UDLPROBLEM["UDL2"]["24boxes"][2]

In [ ]:
contenedor =np.asanyarray( P2A2[0],dtype=np.int64)
datos =np.asanyarray(P2A2[1],dtype=np.int64)
gamma=0.0004
Maxitr = 500
n = len(datos)

In [ ]:
IniPob = CreatePoblation(10, datos)
fireflyPob = InstancePob(NumbaList(IniPob),NumbaList(datos),NumbaList(contenedor))
DFFtrain(1,NumbaList(fireflyPob),gamma,datos,contenedor)

In [16]:
IniPob = CreatePoblation(50, datos)
fireflyPob = InstanceFFn(NumbaList(IniPob),NumbaList(datos),NumbaList(contenedor))
bestff = Train(500,fireflyPob,gamma)

In [ ]:
IniPob = CreatePoblation(50, datos)
fireflyPob = InstanceFFn(NumbaList(IniPob),NumbaList(datos),NumbaList(contenedor))
bestff = Train(500,fireflyPob,gamma)